In [13]:
import geopandas as gpd
import rasterio

tif_path = "./landscan/landscan-global-2015.tif"
geo_path = "./landscan/harbin2.geojson"

# 读取边界文件（Shapefile 或 GeoJSON）
boundary = gpd.read_file(geo_path)

# 读取栅格数据并获取其坐标系
with rasterio.open(tif_path) as src:
    raster_crs = src.crs

# 如果坐标系不同，则转换 GeoJSON 文件
if boundary.crs != raster_crs:
    boundary = boundary.to_crs(raster_crs)
    print("GeoJSON 文件已转换为栅格数据的坐标系:", boundary.crs)

print("边界文件坐标系:", boundary.crs)
print("栅格文件坐标系:", raster_crs)

边界文件坐标系: EPSG:4326
栅格文件坐标系: EPSG:4326


In [14]:
from rasterio.mask import mask
# 裁剪栅格数据
with rasterio.open(tif_path) as src:
    out_image, out_transform = mask(src, boundary.geometry, crop=True)
    out_meta = src.meta.copy()

# 更新元数据
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# 保存裁剪后的栅格
output_tif = "clipped_harbin_landscan_data.tif"
with rasterio.open(output_tif, "w", **out_meta) as dest:
    dest.write(out_image)

In [18]:
from rasterio.transform import xy
tif_path = "clipped_harbin_landscan_data.tif"
with rasterio.open(tif_path) as src:
    population_data = src.read(1)  # 读取第一波段（人口密度）
    transform = src.transform     # 获取仿射变换
    crs = src.crs

# 

# 准备数据
data = []
rows, cols = population_data.shape
for row in range(rows):
    for col in range(cols):
        value = population_data[row, col]
        if value > 0:  # 过滤无效值
            lon, lat = xy(transform, row, col, offset='center')
            data.append((f'SRID=4326;POINT({lon} {lat})', int(value)))
            print(value)

print(len(data))

202
25
87
161


KeyboardInterrupt: 

In [11]:
print(len(data))

3581


In [19]:
import sqlite3

# 创建或连接到名为 "landscan_harbin.db" 的 SQLite 文件
conn = sqlite3.connect("landscan_harbin.db")

# 创建游标对象，用于执行 SQL 语句
cur = conn.cursor()

# 创建数据表
cur.execute("""
CREATE TABLE IF NOT EXISTS harbinpp (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    longitude REAL,
    latitude REAL,
    population_density INTEGER
)
""")

# 提交更改并关闭游标
conn.commit()
cur.close()

print("SQLite 数据库文件已创建并包含表结构。") 

SQLite 数据库文件已创建并包含表结构。


In [20]:
import rasterio
from rasterio.transform import xy

# 重新连接数据库
conn = sqlite3.connect("landscan_harbin.db")
cur = conn.cursor()

# 读取裁剪后的 LandScan 数据
tif_path = "clipped_harbin_landscan_data.tif"
with rasterio.open(tif_path) as src:
    population_data = src.read(1)  # 读取第一波段
    transform = src.transform

# 准备数据
data = []
rows, cols = population_data.shape

for row in range(rows):
    for col in range(cols):
        value = population_data[row, col]
        if value > 0:  # 过滤无效值
            lon, lat = xy(transform, row, col, offset='center')
            data.append((lon, lat, int(value)))

# 批量插入数据
cur.executemany("INSERT INTO harbinpp (longitude, latitude, population_density) VALUES (?, ?, ?)", data)
conn.commit()

# 关闭连接
cur.close()
conn.close()

print("数据已成功插入 SQLite 数据库文件！")

数据已成功插入 SQLite 数据库文件！


In [25]:
import psycopg2

# 数据库连接参数
db_params = {
    'dbname': 'harbintrips',  # 连接到默认的 postgres 数据库
    'user': 'username',   # 替换为你的用户名
    'password': 'password', # 替换为你的密码
    'host': 'localhost',
    'port': '5432'
}

# 创建连接
try:
    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    # 创建表 SQL 语句
    create_table_query = """
    CREATE TABLE IF NOT EXISTS harbinpp (
        id SERIAL PRIMARY KEY,               -- 自动递增的主键
        longitude DOUBLE PRECISION NOT NULL, -- 经度（地理坐标）
        latitude DOUBLE PRECISION NOT NULL,  -- 纬度（地理坐标）
        population_density INTEGER NOT NULL,  -- 人口密度
        timestamp BIGINT          -- 数据时间版本（时间戳）
    );
    """

    # 执行创建表的 SQL
    cur.execute(create_table_query)
    conn.commit()

    print("表 'harbinpp' 创建成功！")

except Exception as e:
    print("发生错误：", e)

finally:
    # 关闭连接
    if cur:
        cur.close()
    if conn:
        conn.close()


表 'harbinpp' 创建成功！


In [29]:
# 读取裁剪后的 LandScan 数据
tif_path = "clipped_harbin_landscan_data.tif"
with rasterio.open(tif_path) as src:
    population_data = src.read(1)  # 读取第一波段
    transform = src.transform

# 准备数据
data = []
rows, cols = population_data.shape

for row in range(rows):
    for col in range(cols):
        value = population_data[row, col]
        if value > 0:  # 过滤无效值
            lon, lat = xy(transform, row, col, offset='center')
            data.append((lon, lat, int(value), 1420070400000))


import psycopg2

# 数据库连接参数
db_params = {
    'dbname': 'harbintrips',  # 连接到默认的 postgres 数据库
    'user': 'username',   # 替换为你的用户名
    'password': 'password', # 替换为你的密码
    'host': 'localhost',
    'port': '5432'
}

# 创建连接
try:
    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    # 插入数据，并显式指定时间戳
    insert_query = """
    INSERT INTO harbinpp (longitude, latitude, population_density, timestamp)
    VALUES (%s, %s, %s, %s);
    """
    for record in data:
        cur.execute(insert_query, record)
        conn.commit()  # 提交每一条插入


    print("数据插入成功！")

except Exception as e:
    print("发生错误：", e)

finally:
    # 关闭连接
    if cur:
        cur.close()
    if conn:
        conn.close()

数据插入成功！


In [ ]:
###
# 45.40416666666613  45.41249999999946  0.008333333 45.4 
# 46.09583333333279  46.08749999999946  0.008333333 46.1
# 127.05416666666545 127.04583333333213 0.008333333 127.058333333
# 126.0874999999988  126.09583333333212 0.008333333 126.083333333
###

# (45.40416666666613 - 45.4)/0.008333333
# (46.09583333333279 - 45.40416666666613) / 0.008333333 = 83
# (127.05416666666545 - 126.0874999999988) / 0.008333333 = 116

# 24*60/10 = 144
# 横向以空间坐标为划分，纵向以时间每10分钟为划分 做聚类

# UPDATE harbinpp
# SET lon_index = CEIL((longitude - 126.083333333) / 0.008333333),
#     lat_index = CEIL((latitude - 45.4) / 0.008333333);
import psycopg2

lat_index = 1
lat1 = 45.4
lat2 = 45.4 + 0.008333333 * lat_index

lon_index = 1
lon1 = 126.083333333 + 0.008333333 * lon_index

#1420243200  1.03
#1420329600  1.04
#1420416000  1.05
#1420502400  1.06
#1420588800  1.07

